In [1]:
import numpy
import pandas
import random
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

In [2]:
#Load all the data---should take around a 1.5 minutes. Files names have been renamed to generics for confidentiality reasons.
xls = pandas.ExcelFile('Data.xlsx')

df1 = pandas.read_excel(xls, '8.8.2023-10.7.2023')
df2 = pandas.read_excel(xls, '10.8.2023-12.7.2023')
df3 = pandas.read_excel(xls, '12.8.2023-1.7.2024')
df4 = pandas.read_excel(xls, '1.8.2024-3.7.2024')
df5 = pandas.read_excel(xls, '3.8.2024-5.7.2024')
df6 = pandas.read_excel(xls, '5.8.2024-8.7.2024')

pt_data = pandas.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)[['Order #', 'item_number', 'tran_qty']]

In [4]:
#Store key values of the orders
number_of_items = pt_data['item_number'].unique().size
number_of_transactions = pt_data['Order #'].unique().size
number_of_transaction_lines = pt_data['Order #'].size

print("Number of Items: " + str(number_of_items))
print("Number of Transactions: " + str(number_of_transactions))
print("Number of Transaction Lines: " + str(number_of_transaction_lines))

Number of Items: 207570
Number of Transactions: 2881211
Number of Transaction Lines: 5272937


In [5]:
#Converts all Item IDs and Transaction IDs to integer values and creates a key
#This step is nessecary because Sparse Matrices can only proccess numerics

item_ids = pt_data['item_number'].unique().tolist()
transaction_ids = pt_data['Order #'].unique().tolist()

def generate_unique_id(existing_ids, limit):
    while True:
        new_id = random.randint(0, limit-1)
        if new_id not in existing_ids:
            existing_ids.add(new_id)
            return new_id
        
def format_qty(quantity):
    if quantity >= 1:
        return 1
    return 0

existing_item_ids = set()
existing_transaction_ids = set()


item_ids_key = {x: generate_unique_id(existing_item_ids, number_of_items) for x in item_ids}
transaction_ids_key = {x: generate_unique_id(existing_transaction_ids, number_of_transactions) for x in transaction_ids}

pt_data['item_number'] = pt_data['item_number'].apply(lambda x: item_ids_key[x])
pt_data['Order #'] = pt_data['Order #'].apply(lambda x: transaction_ids_key[x])
pt_data['tran_qty'] = pt_data['tran_qty'].apply(format_qty)

pt_data.rename(columns={"Order #": "order", "tran_qty": "qty"}, inplace=True)

In [7]:
#Conversion to Sparse Matrix Format
sparse_matrix = csr_matrix((pt_data.qty, (pt_data.order, pt_data.item_number)), shape=(number_of_transactions, number_of_items))
sparse_matrix_coo = sparse_matrix.tocoo()
mask = numpy.isnan(sparse_matrix_coo.data)
sparse_matrix_coo.data[mask] = 0 
sparse_matrix_coo.data = numpy.where(sparse_matrix_coo.data > 0, 1, 0)
sparse_matrix = sparse_matrix_coo.tocsr()
df = pandas.DataFrame.sparse.from_spmatrix(sparse_matrix)  

In [9]:
#Basket Analysis Calulations---keep the min_support value low because of the very high transaction volume.
frequent_itemsets = fpgrowth(df, min_support=0.0001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
rules = rules.sort_values(by='confidence', ascending=False, ignore_index=True)

/Users/tarunshah/Desktop/Venv VS/.venv/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [10]:
#Convert Integer IDs back into the original item IDs
swapped_items_key = {v: k for k, v in item_ids_key.items()}

def translate_ids(x):
    ids = []
    for val in x:
        try:
            ids.append(swapped_items_key[int(val)])
        except:
            ids.append('Unlocatable ID')    
    return(ids)

rules['antecedents'] = rules['antecedents'].apply(translate_ids)
rules['consequents'] = rules['consequents'].apply(translate_ids)
rules['Transactions Per Year'] = rules['support'].apply(lambda x: int(x*number_of_transactions)) 

In [11]:
#Prune redudant baskets---for large baskets, the algorithm will also identify every possible permutation of smaller baskets
#This can make your final dataset explode from 10,000 useful baskets to millions of redudant baskets. Pruning removes these redundancies
rules['itemset'] = rules.apply(lambda row: frozenset(row['antecedents']).union(row['consequents']), axis=1)
rules = rules.sort_values(by='itemset', key=lambda x: x.str.len(), ascending=False)

seen_itemsets = set()
indices_to_keep = []

for index, row in rules.iterrows():
    itemset = row['itemset']
    if not any(itemset < seen_itemset for seen_itemset in seen_itemsets):
        indices_to_keep.append(index)
        seen_itemsets.add(itemset)

exportable_dataset = rules.loc[indices_to_keep]
exportable_dataset.reset_index(drop=True, inplace=True)
exportable_dataset.drop(columns=['itemset'], inplace=True)
exportable_dataset = exportable_dataset[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'Transactions Per Year']]
exportable_dataset.to_csv('PartsTown Year-Long Analysis---Pruned Redundancy Dataset.csv')
